# 前処理  
訓練データには多くの欠損値、データ型がバラバラなど、うまく学習させることができない要因が多い。  
欠損値の補填、新たな特徴の抽出、データ型の統一を行う。  
使用するデータセットはkaggleのTitanic: Machine Learning from Disasterのtrain.csv  
https://www.kaggle.com/c/titanic

In [41]:
import pandas as pd
import numpy as np
from collections import Counter

In [42]:
# 訓練データ
train = pd.read_csv('./dataset/train.csv')

## 特徴量
PassengerId 乗客ID, Survived 生存, Pclass 階級, Name 名前, Sex 性別, Age 年齢, SibSp 夫婦・兄弟, Parch 親・子  
Ticket　チケット番号, Fare 運賃, Cabin 客室番号, Embarked 寄港

In [43]:
# 最初の訓練データ
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [44]:
# それぞれの欠損値の数
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

欠損値は、年齢に177個, 客室番号に687個, 寄港地に2個あることがわかる。

## 特徴の追加

### 敬称(Title)の追加

全ての名前(Name)には、敬称(Mr., Mrs., Miss. …)が付いていることがわかる。  
敬称は、その人の肩書きや職業の社会的地位、性別、大体の年齢など、その人がどういう人なのかを簡単に表したもの。  
名前は全員が異なるので使用できないが、敬称には何かしらパターンがあるのではないかと考える。

In [45]:
# 名前(Name)から敬称(Title)を抽出
train['Title'] = train['Name'].str.extract('([A-Za-z]+)\.', expand=False)

In [46]:
Counter(train['Title'])

Counter({'Capt': 1,
         'Col': 2,
         'Countess': 1,
         'Don': 1,
         'Dr': 7,
         'Jonkheer': 1,
         'Lady': 1,
         'Major': 2,
         'Master': 40,
         'Miss': 182,
         'Mlle': 2,
         'Mme': 1,
         'Mr': 517,
         'Mrs': 125,
         'Ms': 1,
         'Rev': 6,
         'Sir': 1})

敬称には  
Cap:船長, Col:少佐・大佐, Countess:女伯爵, Don:貴人・高位聖職者, Dr:医者・博士号, Jonkheer:男爵, Lady:貴婦人 ,Major:少佐, Master:少年, Miss:未婚女性, Mlle:未婚女性, Mme:既婚女性, Mr:成人男性, Mrs:既婚女性, Ms:女性, Rev:牧師, Sir:騎士  
と意外と種類の多いことがわかる。  　

敬称は社会的地位や年齢を表していることから、年齢の補填にいいのではないか。  
同じ敬称の平均値を補填してみる。

In [47]:
# 年齢に欠損のある敬称
Counter(train[train['Age'].isnull()].Title)

Counter({'Dr': 1, 'Master': 4, 'Miss': 36, 'Mr': 119, 'Mrs': 17})

運のいいことに年齢に欠損値がある人は、同じ敬称の人がいるものであった。

In [48]:
# 欠損値のある敬称の平均値
Dr_age = train.query('Title == "Dr"').Age.mean()
Master_age = train.query('Title == "Master"').Age.mean()
Miss_age = train.query('Title == "Miss"').Age.mean()
Mr_age = train.query('Title == "Mr"').Age.mean()
Mrs_age = train.query('Title == "Mrs"').Age.mean()

In [49]:
# 同敬称の年齢の平均値を補填
train.loc[(train["Title"].values == "Dr") & (train["Age"].isnull()), "Age"] = Dr_age
train.loc[(train["Title"].values == "Master") & (train["Age"].isnull()), "Age"] = Master_age
train.loc[(train["Title"].values == "Miss") & (train["Age"].isnull()), "Age"] = Miss_age
train.loc[(train["Title"].values == "Mr") & (train["Age"].isnull()), "Age"] = Mr_age
train.loc[(train["Title"].values == "Mrs") & (train["Age"].isnull()), "Age"] = Mrs_age

敬称には、MrやMrsなど一般的なものから、MlleやJonkheerなどヨーロッパの貴族の敬称がある。  
名前よりも種類は減ったものの、一人や数人しかいないものもあるので、近い敬称とそれ以外にする。  
残す敬称は、少年(Master), 未婚女性(Miss), 既婚女性(Mrs), 男性(Mr), その他(Others)

In [50]:
# 近い敬称をまとめる
train['Title'] = train['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Countess', 'Sir', 'Jonkheer', 'Lady', 'Capt', 'Don'], 'Others')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')
train['Title'] = train['Title'].replace('Mlle', 'Miss')

### グループ人数  
救命ボートに乗る際、複数人のグループで乗るか、一人で乗るかによって、生存に影響が出るのではないかと考えた。  
例えば、一人や少人数であれば満員の救命ボートに潜り込める。一方で複数人グループで誰かが乗れないと、一緒に降りるといったことがあったかもしれない。  
  
元のデータには、夫婦・兄弟、親・子の特徴があったので、これよりグループの人数を求める。

In [51]:
# 特徴「家族の人数」を追加
train['GroupSize'] = train['SibSp'] + train['Parch'] + 1

In [52]:
# 補填と特徴を追加した訓練データ
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,GroupSize
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,1


## 必要ない特徴の除外、欠損値の補填、数値化  
乗客ID, 名前, 夫婦・兄弟の数, 親・子の数, チケット番号, 客室番号を除外  
寄港地の欠損値はSを補填  
性別・寄港地・敬称をそれぞれ数値化  

In [53]:
train = train.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis = 1)
train = train.fillna({'Embarked' :'S'})
train = train.replace({
    'male' : '0', 'female' : '1', 
    'S' : '0', 'C' : '1', 'Q' : '2',
    'Master' : '0', 'Miss' : '1', 'Mrs' : '2', 'Mr' : '3', 'Others' : '4'
})

In [56]:
#前処理が完了した訓練セット
train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,GroupSize
0,0,3,0,22.0,7.2500,0,3,2
1,1,1,1,38.0,71.2833,1,2,2
2,1,3,1,26.0,7.9250,0,1,1
3,1,1,1,35.0,53.1000,0,2,2
4,0,3,0,35.0,8.0500,0,3,1
